## Using Flux Balance Analysis to Compute the Open Extent of Reaction and Fractional Conversion

<figure>
<img src="figs/Fig-FBA-ToyNetwork.pdf" style="width:40%">
</figure>

__Fig 1__. The starting compound $A_{1}$ is converted to the desired product $P$ through an intermediate $B$. The conversion of $A_{1}$ to $B$ requires the cofactor $x$. The cofactor $x$ can be recycled by converting $A_{2}$ to the by-product $C$.

### Introduction

Suppose we want to produce the desired product _P_ by converting feedstock A₁ using a cell-free biochemical process operating in a well-mixed continuous microfluidic chip with two input channels and a single output channel, and a liquid reaction volume of V = 100 μL. The reaction network is shown in Fig 1. 

All the reactions are enzyme-catalyzed and irreversible, where the enzymes are tethered to the chip. The enzymes are present at $E_{\star}$ = 100.0 $\mu$mol/L. The $k_{cat}$ values for the enzymes 1, 2 and 3 are 85.7 $\text{min}^{-1}$, 38.1 $\text{min}^{-1}$ and 13.7 $\text{min}^{-1}$, respectively. Syringe pumps power stream 1 and 2 with a maximum volumetric flow rate of $F_{\star}$ = 10 mL/h. Stream 1 comes from an infinite reservoir containing $A_{1}$ at 1 mmol/L. Likewise, Stream 2 comes from an infinite reservoir containing $A_{2}$ at 0.1 mmol/L.

__Assumptions__
* Microfluidic chip is well-mixed and operates at steady-state
* Constant T, P on the chip and the liquid phase is ideal

__Compute__
* Compute the optimal open extent of reaction $\dot{\epsilon}_{i}$, where that maximizes _P_
* Compute the state table for the mol flow rates into and from the chip
* Compute the fractional conversion of the feedstock A₁

In [13]:
import Pkg; Pkg.activate("."); Pkg.resolve(); Pkg.instantiate();

  Activating project at `~/Desktop/julia_work/ENGRI-1120-IntroToChemE-Example-Notebooks/notebooks-jupyter/ENGRI-1120-Example-Matrix-Vector-System`
  No Changes to `~/Desktop/julia_work/ENGRI-1120-IntroToChemE-Example-Notebooks/notebooks-jupyter/ENGRI-1120-Example-Matrix-Vector-System/Project.toml`
  No Changes to `~/Desktop/julia_work/ENGRI-1120-IntroToChemE-Example-Notebooks/notebooks-jupyter/ENGRI-1120-Example-Matrix-Vector-System/Manifest.toml`


In [37]:
using PrettyTables
using GLPK

┌ Info: Precompiling GLPK [60bf3e95-4087-53dc-ae20-288a0d20c6a6]
└ @ Base loading.jl:1662


In [2]:
# load my code library -
include("Include.jl");

### Before we do anything: Establish a Consistent Unit System
We have different units, let's pick a system and convert the required values to that system. Let's use minutes for time, $\mu$mol for quantity and L for volume.

In [80]:
# convert some units -
V = 100.0*(1e-6);            # system volume in L
F_max = 10.0*(1/60)*(1/1e3); # maximum pump rate in L/min
A_1_in = 1.0*(1e6/1e3);      # concentration of A₁ in input tank (μmol/L)
A_2_in = 0.1*(1e6/1e3);      # concentration of A₂ in input tank (μmol/L)

## a) Compute the optimal open extent of reaction $\dot{\epsilon}_{i}$
We will use [Linear Programming (LP)](https://en.wikipedia.org/wiki/Linear_programming) to estimate an optimal values for the open extents of reaction. To solve the LP problem, we'll need:

* The stoichiometric matrix $S$
* The input feed vectors $\dot{n}_{1}$ and $\dot{n}_{2}$
* Bounds on the permssible values of the reaction rates

#### Linear Programming Problem Setup (Advanced topic)
We know from class that the open mole balance around component $i$ for the chip (at steady-state) is given by:

$$\sum_{s\in\mathcal{S}}\nu_{s}\dot{n}_{s,i} + \sum_{j\in\mathcal{R}}\sigma_{ij}\dot{\epsilon}_{j} = 0\qquad{i=1,2,\dots,\dim(\mathcal{M})}$$

The first term is the rate of `transport` into and from the control volume (units: mol $i$/time) from $\mathcal{S}$ possible `streams`; $v_{s} = 1$ if the stream $s$ enters control volume, $v_{s}=-1$ is stream $s$ exits the control volume. The second summation denotes the reaction terms, where $\sigma_{ij}$ denote the stoichiometric coefficient describing the connection between metabolite $i$ and reaction $j$ and $\dot{\epsilon}_{j}$ denote the open extent of reaction (units: mol/time). For the chip we have two inputs stream entering (s=1 and s=2), and a single stream exiting (s=3) the chip. In this case, the open mole balance becomes:

$$\dot{n}_{3,i} = \dot{n}_{1,i} + \dot{n}_{2,i}+ \sum_{j\in\mathcal{R}}\sigma_{ij}\dot{\epsilon}_{j}\qquad{i=1,2,\dots,\dim(\mathcal{M})}$$

These balances can be used as constraints to find the optimal open extent of reaction. In particular, we know that we actually pass $\alpha\leq{S\dot{\epsilon}}\leq\beta$ to the solver. Thus, because $\dot{n}_{i,3}\geq{0}$, the linear programming problem is subject to the mol constraints: 

$$\dot{n}_{1,i} + \dot{n}_{2,i} + \sum_{j\in\mathcal{R}}\sigma_{ij}\dot{\epsilon}_{j}\geq{0}\qquad\forall{i}$$

In other words, when searching for the optimal set of $\dot{\epsilon}_{j}$ we have to select values that give physically realistic answers (we can't have a negative mol flow rate). Next, the $\dot{\epsilon}_{j}$ terms (just flux) are bounded from above and below: 

$$\mathcal{L}_{j}\leq\dot{\epsilon}_{j}\leq\mathcal{U}_{j}\qquad{j=1,2\dots,\dim(\mathcal{R})}$$

where the $\mathcal{L}_{j}$ and $\mathcal{U}_{j}$ denote the lower and upper bounds that $\dot{\epsilon}_{j}$ can take, remember that the open extents $\dot{\epsilon}_{j}$ are just reaction rates times the volume. Thus, the lower and upper bounds describe the permissible range we expect the rate _could_ obtain.  

Putting everything together gives a problem formulation to compute the mol/time flux through a reaction network. An objective:

$$\min_{\dot{\epsilon}}\sum_{j\in\mathcal{R}}c_{j}\dot{\epsilon}_{j}$$

is minimized (or maximized) subject to a collection of linear constraints:

$$\begin{eqnarray}
\sum_{j\in\mathcal{R}}\sigma_{ij}\dot{\epsilon}_{j}&\geq&{-\dot{n}_{1,i}-\dot{n}_{2,i}}\qquad\forall{i}\\ 
\mathcal{L}_{j}&\leq\dot{\epsilon}_{j}\leq&\mathcal{U}_{j}\qquad{j=1,2\dots,\mathcal{R}}
\end{eqnarray}$$

#### Formulate Species and Reaction Sets
Let the species set $\mathcal{M}=\left\{A_{1}, A_{2}, B, C, P, x, y\right\}$ and the reaction set $\mathcal{R}=\left\{r_{1}, r_{2}, r_{3}\right\}$.

In [6]:
# Fill me in ... setup the system dimensions
M = 7; # number of species M = dim(ℳ)
R = 3; # number of reactions R = dim(ℛ)

#### Formulate the Stoichiometric Matrix
The stoichiometric matrix $S$ holds the stoichiometric coefficients; $S$ is a $\dim(\mathcal{M)}\times\dim{\mathcal{R}}$ array where the (i,j) entry, denoted by $\sigma_{ij}$ follows the rules:

* If $\sigma_{ij}>0$ then species $i$ is _produced_ by reaction $j$, i.e., species $i$ is a product of reaction $j$ 
* If $\sigma_{ij}=0$ then species $i$ is _not connected to_ reaction $j$
* If $\sigma_{ij}<0$ then species $i$ is _consumed_ by reaction $j$, i.e., species $i$ is a reactant of reaction $j$.

In [11]:
# Building the stoichiometric matrix is a hassle, so I wrote a routine to do it for us!

# build the list of reactions we want in the S -
reaction_list = Array{String,1}(undef, R)
reaction_list[1] = "r₁,A₁+x,B+y,false" # string for r₁
reaction_list[2] = "r₂,B,P,false"      # string for r₂
reaction_list[3] = "r₃,A₂+y,C+x,false" # string for r₃

# buld the stoichiometric matrix -
(SM, SSA, RSA) = build_stoichiometric_matrix(reaction_list);

#### Formulate the Reaction Constraints
The open extent of reaction $\dot{\epsilon}_{i}$ can be written as:

$$\dot{\epsilon}_{i}=r_{i}V$$

where $r_{i}$ is a [kinetic rate law](https://en.wikipedia.org/wiki/Rate_equation) for reaction $i$ and $V$ denotes the reaction volume. The rate law of an enzyme-catalyzed reaction (approximately) follows the [Michaelis-Menton]( https://en.wikipedia.org/wiki/Michaelis–Menten_kinetics) expression:

$$v = k_{cat}E\left(\frac{S}{K_{m}+S}\right)$$

where $v$ denotes the reaction velocity (or rate, units: mole/L-time), $S$ denotes the substrate of the reaction (units: mole/L), $K_{m}$ denotes the saturation constant (units: mole/L), $k_{cat}$ denotes the catalytic constant (units: 1/time) and $E$ denotes the enzyme concentration (units: mole/L). 

However, our problem is more advanced, as we have reactions with multiple substrates. Thus, we don't know the rate, so we can't get a nice expression of the open extent. Hmmmm.

In [22]:
# setup reaction bounds array -
flux_bounds_array = zeros(R,2);

# what are my kcats?
E = 100.0; # units: μmol/L
k₁ = 85.7; # units: min^-1
k₂ = 38.1; # units: min^-1
k₃ = 13.7; # units: min^-1

# set the upper bounds -
flux_bounds_array[1,2] = k₁*E*V;
flux_bounds_array[2,2] = k₂*E*V;
flux_bounds_array[3,2] = k₃*E*V;

In [46]:
flux_bounds_array

3×2 Matrix{Float64}:
 0.0  0.857
 0.0  0.381
 0.0  0.137

#### Formulate the Species Constraints

In [81]:
# setup stream inputs -
F₁ = 1.0*F_max;
F₂ = 1.0*F_max;

# stream 1 -
ṅ_1 = [
    F₁*A_1_in ; # 1 A₁
    0.0       ; # 2 A₂
    0.0       ; # 3 B
    0.0       ; # 4 C
    0.0       ; # 5 P
    0.0       ; # 6 x
    0.0       ; # 7 y
];

# stream 2 -
ṅ_2 = [
    0.0       ; # 1 A₁
    F₂*A_2_in ; # 2 A₂
    0.0       ; # 3 B
    0.0       ; # 4 C
    0.0       ; # 5 P
    0.0       ; # 6 x
    0.0       ; # 7 y
];

In [82]:
# initialize some space -
species_constraint_array = Array{Float64,2}(undef, M, 2)
for i ∈ 1:M
    species_constraint_array[i,1] = - ṅ_1[i] - ṅ_2[i] # lower bound 
    species_constraint_array[i,2] = 1000.0            # upper bound
end

In [45]:
objective_coefficient_array = [0.0,-1.0,0.0];

In [83]:
# call the flux solver -
results_tuple = flux(SM, flux_bounds_array, species_constraint_array, objective_coefficient_array);

In [92]:
# what is the optimal extent of reaction
ϵ̇ = results_tuple.calculated_flux_array;

# setup -
extent_table_header = (["Reaction index", "ϵ̇ᵢ"], ["","mol/min"])
extent_data_table = Array{Any,2}(undef,R,2);
for i ∈ 1:R
    extent_data_table[i,1] = i
    extent_data_table[i,2] = ϵ̇[i]
end

# show -
pretty_table(extent_data_table; header=extent_table_header)

┌────────────────┬───────────┐
│ Reaction index │        ϵ̇ᵢ │
│                │   mol/min │
├────────────────┼───────────┤
│              1 │ 0.0166667 │
│              2 │ 0.0166667 │
│              3 │ 0.0166667 │
└────────────────┴───────────┘


## b) Formulate the State Table

In [87]:
# what is ṅ_3?
ṅ_3 = ṅ_1 .+ ṅ_2 .+ SM*ϵ̇;

In [86]:
# setup table -
state_table_header = (["Species", "ṅ_1", "ṅ_2", "ṅ_3"], ["", "mol/min", "mol/min", "mol/min"])

# setup array to hold the state data -
state_data_array = Array{Any,2}(undef, M, 4);

# populate the data for the table -
for i ∈ 1:M
    state_data_array[i,1] = SSA[i]
    state_data_array[i,2] = ṅ_1[i]
    state_data_array[i,3] = ṅ_2[i]
    state_data_array[i,4] = ṅ_3[i]
end

# show -
pretty_table(state_data_array; header=state_table_header)

┌─────────┬──────────┬───────────┬───────────┐
│ Species │      ṅ_1 │       ṅ_2 │       ṅ_3 │
│         │  mol/min │   mol/min │   mol/min │
├─────────┼──────────┼───────────┼───────────┤
│      A₁ │ 0.166667 │       0.0 │      0.15 │
│      A₂ │      0.0 │ 0.0166667 │       0.0 │
│       B │      0.0 │       0.0 │       0.0 │
│       C │      0.0 │       0.0 │ 0.0166667 │
│       P │      0.0 │       0.0 │ 0.0166667 │
│       x │      0.0 │       0.0 │       0.0 │
│       y │      0.0 │       0.0 │       0.0 │
└─────────┴──────────┴───────────┴───────────┘


## c) Compute the Fraction Conversion of $A_{1}$

In [95]:
f₁ = (ṅ_1[1] + ṅ_2[1] - ṅ_3[1])/(ṅ_1[1] + ṅ_2[1])

0.09999999999999998